<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-Exploration" data-toc-modified-id="Data-Exploration-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Exploration</a></span><ul class="toc-item"><li><span><a href="#Load-and-check" data-toc-modified-id="Load-and-check-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load and check</a></span></li><li><span><a href="#Correlation-Matrix" data-toc-modified-id="Correlation-Matrix-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Correlation Matrix</a></span></li><li><span><a href="#Plotting-data-distribution" data-toc-modified-id="Plotting-data-distribution-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Plotting data distribution</a></span><ul class="toc-item"><li><span><a href="#Time-series" data-toc-modified-id="Time-series-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Time series</a></span></li></ul></li></ul></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#Plot-feature-importance" data-toc-modified-id="Plot-feature-importance-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Plot feature importance</a></span></li><li><span><a href="#Creating-new-columns" data-toc-modified-id="Creating-new-columns-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Creating new columns</a></span></li><li><span><a href="#Merge-Rare-values" data-toc-modified-id="Merge-Rare-values-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Merge Rare values</a></span></li><li><span><a href="#Hashing-Trick" data-toc-modified-id="Hashing-Trick-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Hashing Trick</a></span></li><li><span><a href="#Recursive-hashing" data-toc-modified-id="Recursive-hashing-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Recursive hashing</a></span></li></ul></li><li><span><a href="#Algorithms" data-toc-modified-id="Algorithms-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Algorithms</a></span><ul class="toc-item"><li><span><a href="#RandSearchCV-XGBoostClassifier" data-toc-modified-id="RandSearchCV-XGBoostClassifier-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>RandSearchCV XGBoostClassifier</a></span></li><li><span><a href="#RandSearchCV-Light-Gradient-Boosting-Machine" data-toc-modified-id="RandSearchCV-Light-Gradient-Boosting-Machine-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>RandSearchCV Light Gradient Boosting Machine</a></span></li><li><span><a href="#Score-evolution-with-1-parameter-variation" data-toc-modified-id="Score-evolution-with-1-parameter-variation-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Score evolution with 1-parameter variation</a></span></li><li><span><a href="#Save-a-trained-model" data-toc-modified-id="Save-a-trained-model-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Save a trained model</a></span></li><li><span><a href="#Plot-learning-curve" data-toc-modified-id="Plot-learning-curve-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Plot learning curve</a></span></li></ul></li><li><span><a href="#Challenges" data-toc-modified-id="Challenges-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Challenges</a></span><ul class="toc-item"><li><span><a href="#Making-a-submission" data-toc-modified-id="Making-a-submission-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Making a submission</a></span></li></ul></li><li><span><a href="#Metrics-and-Data-manipulation" data-toc-modified-id="Metrics-and-Data-manipulation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Metrics and Data manipulation</a></span><ul class="toc-item"><li><span><a href="#Plot-ROC-curve" data-toc-modified-id="Plot-ROC-curve-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Plot ROC curve</a></span></li><li><span><a href="#Evaluate-the-model" data-toc-modified-id="Evaluate-the-model-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Evaluate the model</a></span></li><li><span><a href="#Submissions-table" data-toc-modified-id="Submissions-table-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Submissions table</a></span></li><li><span><a href="#Dataset-reduction-/-optimization" data-toc-modified-id="Dataset-reduction-/-optimization-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Dataset reduction / optimization</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Exploration

50 visualisations with Matplotlib : 
https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/

## Load and check

In [ ]:
# Loading data
xtest = pd.read_csv('file.csv')

# Counting NA values in dataframe
print(xtest.isna().sum().sum())

## Correlation Matrix

In [ ]:
correlations = train.corr()
# plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
plt.show()

## Plotting data distribution

##### Matplotlib
http://www.labri.fr/perso/nrougier/teaching/matplotlib/matplotlib.html

In [ ]:
# Quantitative variables distribution
plt.figure(4, figsize=(15, 9))

for col, n in zip(xtrain.columns[0:14], range(1, 15)):
    
    plt.subplot(4, 4, n)
    plt.xlabel(col)
    
    plt.hist(xtrain[col], label=('x', 'y'))
    plt.draw()

In [ ]:
# Classes histogram
plt.figure(figsize=(15, 3))
plt.hist(digits['target'], bins=19 , color='green')
plt.show()

### Time series

In [ ]:
ias_brut = grippal[['PERIODE','IAS_brut']]

ias_brut.plot(figsize=(15, 4))
plt.xticks(ticks=range(0, ias_brut.shape[0], 500), labels=ias_brut['PERIODE'][::500])
plt.show()

# Feature Engineering

## Plot feature importance

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

plt.figure(figsize=(10,10))

name = "Random Forest"

indices = np.argsort(rf.feature_importances_)[::-1][:40]
g = sns.barplot(y=X_train.columns[indices][:40],x = rf.feature_importances_[indices][:40] , orient='h')
g.set_xlabel("Relative importance",fontsize=12)
g.set_ylabel("Features",fontsize=12)

g.tick_params(labelsize=9)
g.set_title("Feature importance")
plt.savefig('images/importance.png')

plt.show()

## Creating new columns

In [ ]:
xtrain_fe = xtrain.copy()

for column in xtrain_fe.columns:
    new_col = column+'_sq'
    xtrain_fe[new_col] = pow(xtrain_fe[column], 2)

## Merge Rare values

With this function, we assign the value 'isRare' to values appearing less than a X times. X being the threshold of our function.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MergeRareTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, col_names, threshold):
        self.col_names = col_names
        self.threshold = threshold

    def fit(self, X, y=None):

        X = pd.DataFrame(X)
        counts_dict_list = []
        
        for i in range(len(self.col_names)):
            
            serie = X[self.col_names[i]].value_counts()  
            rare_indexes = serie[serie<self.threshold[i]].index 
            frequent_indexes = serie[serie>=self.threshold[i]].index  
            dico = {x:'isRare' for x in rare_indexes}
            dico.update({x: str(x) for x in frequent_indexes})
            counts_dict_list.append(dico) 
            
        self.counts_dict_list_ = counts_dict_list
        return self

    def transform(self, X):

        Xt = pd.DataFrame()
        for col, count_dict in zip(self.col_names, self.counts_dict_list_):
            Xt[col] = X[col].apply(lambda x:count_dict[x] if x in count_dict else 'isRare')

        return Xt

In [ ]:
# Applying function
mg = MergeRareTransformer(col_names=Xtrain.columns, threshold=[20]*len(Xtrain.columns))

Xtrain_mg = mg.fit_transform(Xtrain)
Xtest_mg = mg.transform(Xtest)

## Hashing Trick

In [ ]:
import hashlib
# Create new dataframes
Xtrain_ha = pd.DataFrame()
Xtest_ha = pd.DataFrame()

# Apply hashing 
for col in Xtrain_mg.columns:
    Xtrain_ha[col] = Xtrain_mg[col].apply(lambda x: hash(str(x)) % 1000000) # Fill here defining a lambda function
    Xtest_ha[col] = Xtest_mg[col].apply(lambda x: hash(str(x)) % 1000000)

## Recursive hashing

In [ ]:
# Continue with hashed dataframes
Xtrain_haha = pd.DataFrame(Xtrain_ha).copy()
Xtest_haha = pd.DataFrame(Xtest_ha).copy()

n_hash = 3
cols = Xtrain_ha.columns
for l in range(n_hash):
    for col in cols:
        Xtrain_haha[col + '-hash'] = Xtrain_haha[col].apply(lambda x:hash(str(x)))
        Xtest_haha[col + '-hash'] = Xtest_haha[col].apply(lambda x:hash(str(x)))
    cols = [col + '-hash' for col in cols]

# Algorithms

## RandSearchCV XGBoostClassifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

# Classifier
xgb = XGBClassifier(
    n_jobs=-1,
    silent=False)

# Create hyperparameter options
xgb_max_depth=[3, 5, 7, 10]             # Usual values between 3-10
xgb_learning_rate=[0.1, 0.5, 1, 1.2]    # Makes the model more robust by shrinking the weights on each step
xgb_n_estimators=[100, 200, 500, 1000, 1200]
xgb_booster=['gbtree']                  #, 'gblinear', 'dart']
xgb_reg_lambda=[1, 2]                   # L2 used to reduce overfitting

hyperparameters = dict(
    max_depth = xgb_max_depth, 
    learning_rate = xgb_learning_rate,
    n_estimators = xgb_n_estimators,
    booster=xgb_booster,
    reg_lambda=xgb_lambda)

# Create randomized grid search
rscv = RandomizedSearchCV(xgb, hyperparameters, random_state=1, n_iter=50, cv=5, verbose=10, n_jobs=-1)
# Fit randomized search
best_model = rscv.fit(xtrain_part, ytrain_part)

# View Hyperparameter Values Of Best Model
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best learning_rate:', best_model.best_estimator_.get_params()['learning_rate'])
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])

## RandSearchCV Light Gradient Boosting Machine

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb

xtrain_lgbm = xtrain.head(n=100000)
ytrain_lgbm = ytrain.head(n=100000)

# Classifier
lgb_estimator = lgb.LGBMClassifier(
    boosting_type='gbdt',  
    objective='binary', 
    n_estimators=100, 
    learning_rate=0.1, 
    metric='binary_logloss',
    n_jobs=-1)

# Create hyperparameter options
hyperparameters = dict(
    num_leaves = [31, 60, 128, 160, 250], 
    reg_alpha = [0.1, 0.5],
    min_data_in_leaf = [30, 50, 100, 300, 400],
    n_estimators = [100, 200, 500, 1000, 2000, 5000]
    lambda_l1 = [0, 1, 1.5],
    lambda_l2 = [0, 1],
    max_depth = [7],
    )

# Create randomized grid search
rscv = RandomizedSearchCV(
    lgb_estimator, hyperparameters,
    n_iter=100, cv=5, 
    verbose=20, n_jobs=-1)

# Fit randomized search
best_model = rscv.fit(xtrain_lgbm, np.ravel(ytrain_lgbm))

# View Hyperparameter Values Of Best Model
print('Best num_leaves:', best_model.best_estimator_.get_params()['num_leaves'])
print('Best reg_alpha:', best_model.best_estimator_.get_params()['reg_alpha'])
print('Best min_data_in_leaf:', best_model.best_estimator_.get_params()['min_data_in_leaf'])
print('Best lambda_l1:', best_model.best_estimator_.get_params()['lambda_l1'])
print('Best lambda_l2:', best_model.best_estimator_.get_params()['lambda_l2'])

## Score evolution with 1-parameter variation
For this snippet, data needs to be train-test splitted.

In [ ]:
# For DecisionTree
accuracy_gini = []
accuracy_entropy = []
depth = [1, 2, 4, 6, 8, 10, 11, 12, 13, 14, 15, 20]
for i in depth:
    class_tree = tree.DecisionTreeClassifier(criterion='gini', max_depth=i)
    class_tree.fit(x1_train, y1_train)
    y1_pred = class_tree.predict(x1_test)
    accuracy_gini.append(metrics.accuracy_score(y1_test, y1_pred, normalize=False))
    
    class_tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i)
    class_tree.fit(x1_train, y1_train)
    y1_pred = class_tree.predict(x1_test)
    accuracy_entropy.append(metrics.accuracy_score(y1_test, y1_pred, normalize=False))

plt.figure(figsize=(15, 4))
plt.plot(depth, accuracy_gini)
plt.plot(depth, accuracy_entropy)
plt.xlabel("Paramètre MaxDepth")
plt.ylabel("Accuracy")
plt.legend(["gini", "entropy"])
plt.show()

In [ ]:
# For XGBC
accuracy_one = []
accuracy_two = []
depth = [1, 2, 4, 6, 7, 8, 10, 11, 12]

for i in depth:
    print("Round "+str(i))
    # Model 1
    xgb1 = XGBClassifier(n_jobs=-1, max_depth=i, n_estimators=10)
    xgb1.fit(X_train, np.ravel(y_train))
    y1_pred = xgb1.predict(X_test)
    accuracy_one.append(accuracy_score(y_test, y1_pred, normalize=False))
    
    # Model2
    xgb2 = XGBClassifier(n_jobs=-1, max_depth=i, n_estimators=50)
    xgb2.fit(X_train, np.ravel(y_train))
    y1_pred = xgb2.predict(X_test)
    accuracy_two.append(accuracy_score(y_test, y1_pred, normalize=False))

plt.figure(figsize=(15, 4))
plt.plot(depth, accuracy_one)
plt.plot(depth, accuracy_two)
plt.xlabel("Paramètre MaxDepth")
plt.ylabel("Accuracy")
plt.legend(["100 estimators", "200 estimators"])
plt.show()

plt.savefig('images/max_depth_XGB.png')

## Save a trained model
Source : https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [ ]:
import pickle

model = LogisticRegression()
model.fit(X_train, Y_train)

# Save the model to disk
filename = 'saved_lr.sav'
pickle.dump(model, open(filename, 'wb'))
 
# Load the model from disk
filename = 'saved_lr.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)

def save_model(model, filename):
    pickle.dump(model, open(filename, 'wb'))
    
def load_model(model, filename):
    loaded_model = pickle.load(open(filename, 'rb'))
    return pickle.load(open(filename, 'rb'))

## Plot learning curve

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure(figsize=(15, 4))
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

class_learn = tree.DecisionTreeClassifier(criterion='entropy', max_depth=18)
class_learn.fit(data_train, target_train)

plot_learning_curve(class_learn, 'Classifier Learning Curve', digits.data, digits.target)
plt.show()

# Challenges

## Making a submission

In [4]:
def make_submission(test_data, algorithm, filename='submission.csv'):
    """Creates a CSV file for challenge submission
  
    test_data: Description of arg1 
    algorithm: Algo used for making prediction
    filename: 'submission.csv'
    """
    ytest = algorithm.predict(test_data)
    np.savetxt(fichier, ytest, fmt = '%1.0d', delimiter=',')

# Metrics and Data manipulation

## Plot ROC curve

In [ ]:
from sklearn.metrics import roc_curve, auc

def plot_roc_curve(ytest, soft):

    fpr, tpr, threshold = roc_curve(ytest, soft[:,1])
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(15, 3))
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
plot_roc_curve(y_test, lgbm.predict_proba(X_test_fe))

## Evaluate the model

In [ ]:
def evaluate_model(name, model, xtrain, ytrain, xtest, ytest):
    
    print("Fitting..")
    lgbm.fit(xtrain, np.ravel(y_train))

    print("Predicting..")
    soft = lgbm.predict_proba(xtest)
    hard = lgbm.predict(xtest)
    
    print("Scoring :")
    print(name, " ", accuracy_score(ytest, hard, normalize=True, sample_weight=None))

## Submissions table
Fill a score dictionnary along the way, and finally display it as a frame.

In [3]:
lgbm_submissions = {}
lgbm_submissions['1'] = [500, 0.1, -1, 0.98465]
lgbm_submissions['2'] = [3000, 0.08, -1, 0.985015830111]
lgbm_submissions['3'] = [5000, 0.01, -1, 0.987662524559]

lgbm_subs_df = pd.DataFrame.from_dict(
    lgbm_submissions, 
    orient='index',
    columns=['n_estimators', 'learning_rate', 'num_leaves', 'score'])

lgbm_subs_df

,n_estimators,learning_rate,num_leaves,score
1,500,0.10,-1,0.984650
2,3000,0.08,-1,0.985016
3,5000,0.01,-1,0.987663


## Dataset reduction / optimization

Fit a KNNClassifier on the training set, and find neighbors in the test set. Only keep these neighbors. This technique can filter out training samples to make it more similar to the testing set.

In [4]:
from sklearn.neighbors import KNeighborsClassifier

def optimize_dataset(Xtrain, Ytrain, Xtest, n_neighbors=3, save_csv=False):
    
    knn = KNeighborsClassifier(
        n_neighbors=3,
        n_jobs=-1)

    print('Fitting KNNClassifier')
    knn.fit(Xtrain, np.ravel(Ytrain))

    # Find indices of similar points.
    neighbors_idx_list = np.unique(np.ravel(knn.kneighbors(xtest)[1]))
    
    # Filter out with indices
    xtrain_opt = Xtrain.iloc[neighbors_idx_list, :]
    ytrain_opt = Ytrain.iloc[neighbors_idx_list, :]

    print("Now we have ", xtrain_opt.shape[0], " lines for our training set.")
    
    if save_csv:
        print("Writing CSV files : x_optim.csv / y_optim.csv")
        xtrain_opt.to_csv('x_optim.csv')
        ytrain_opt.to_csv('y_optim.csv')